# Introduction

This notebook will be used for Webscrapping and Clustering neighborhoods in Toronto, Canada. It has 3 parts:

## Part 1:
Scrape Toronto Neighborhood Data from Wikipedia
## Part 2:
Read the Latitude and Longitude Coordinates from a csv file and update the dataframe
## Part 3:
Cluster Using K-Means Clustering & Visualize in a Map

## Part 1:
Scrape Toronto Neighborhood Data from Wikipedia

We will now install and import the first set of libraries needed. Including Beautiful Soup and lxml parser for web scrapping

In [ ]:
!pip install html5lib;
!pip install request;
!pip install beautifulsoup4;

In [3]:
import pandas as pd;
import numpy as np;
import requests;
from bs4 import BeautifulSoup;

In [ ]:
Get-retrieve the website content

In [4]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M";
r = requests.get(URL);
print(r)

<Response [200]>


Initialize empty neigbourhood pandas dataframe with the needed columns

In [5]:
column_names = ['PostalCode','Borough', 'Neighborhood']; 
neigbourhood_df = pd.DataFrame(columns=column_names);
neigbourhood_df

,PostalCode,Borough,Neighborhood


Get the object of the BeautifulSoup and read from Wikipedia and print the output

In [6]:
soup = BeautifulSoup(r.content, 'html5lib')

table = soup.body.table.tbody
#print(table);

Parse and Loop through the html while populating the initialized dataframe

In [7]:
for row in table.findAll('tr'):
        #print("Current tds ==", row.findAll('td'));
        tdRow = row.findAll('td');
        if(len(tdRow)!=0):
            borough = str(tdRow[1]).replace("<td>","").replace("</td>","").replace("\n","");
            if(borough.startswith('Not')==True):
                continue
            else:
                postalCode = str(tdRow[0]).replace("<td>","").replace("</td>","").replace("\n","");
                neighbourhood = str(tdRow[2]).replace("<td>","").replace("</td>","").replace("\n","");
                neigbourhood_df=neigbourhood_df.append({'PostalCode':postalCode,
                                'Borough':borough,
                                'Neighborhood':neighbourhood},ignore_index=True);
        
neigbourhood_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [8]:
neigbourhood_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Print out the shape

In [9]:
neigbourhood_df.shape

(103, 3)

## Part 2:
Read the Latitude and Longitude Coordinates from a csv file and update the dataframe

Now we move over to get the Longitude and Latitude of neigbourhood_df from a csv file

In [10]:

neigbour_lat_long_df = pd.read_csv("https://cocl.us/Geospatial_data");

Display the data

In [11]:
neigbour_lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Loop through the neigbourhood_df, get each postalcode and use it to retrive the corresponding coordinates from the neig_lat_long_df

In [12]:
for i in range(0,len(list(neigbourhood_df['PostalCode']))):
    postalCode = neigbourhood_df.loc[i,'PostalCode'];
    postal_df = neigbour_lat_long_df[neigbour_lat_long_df['Postal Code']==postalCode]
    neigbourhood_df.loc[i,'Latitude'] = postal_df['Latitude'].values[0];
    neigbourhood_df.loc[i,'Longitude'] = postal_df['Longitude'].values[0];

display the resultant  and updated neigbourhood_df

In [13]:
neigbourhood_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Part 3:
Cluster Using K-Means Clustering & Visualize in a Map

Lets's install and import the needed libraries

In [ ]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')

Display all the boroughs

In [16]:
neigbourhood_df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

Let's cluster Scarborough, Toronto

Query the dataframe

In [15]:

scarborough = neigbourhood_df[neigbourhood_df['Borough'] == 'Scarborough']
scarborough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
22,M1G,Scarborough,Woburn,43.770992,-79.216917
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Display the shape

In [16]:
scarborough.shape

(17, 5)

Get the geographic coordinates

In [17]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


Lets's visualize Scarborough and the neigbourhoods in it using folium map.

In [19]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(scarborough['Latitude'], scarborough['Longitude'], scarborough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

Define the credentials for FourSquare API used to explore the neighbourhoods around Scarabourough

In [20]:
CLIENT_ID = 'FRDRE5D5LQJ3ZZ2HNIU3DWUBYTN0YCGFX4HUEJPTDR0LXYPU' # API ID
CLIENT_SECRET = 'ICUO2SXLFRCLS1NB24S3YO52TGI4JL4LDBWRVITU4X3HPQYK' # API Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FRDRE5D5LQJ3ZZ2HNIU3DWUBYTN0YCGFX4HUEJPTDR0LXYPU
CLIENT_SECRET:ICUO2SXLFRCLS1NB24S3YO52TGI4JL4LDBWRVITU4X3HPQYK


Define a function to get a nearby venue

In [22]:
def getScaraBouroughNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results]);
        
    # Convert to dataframe
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get alll the nearby venues of the neighbourhoods in Scarborough and assign the result as a dataframe

In [23]:
LIMIT = 100 # first 100 venues

scarborough_nearby_venues = getScaraBouroughNearbyVenues(names=scarborough['Neighborhood'],
                                   latitudes=scarborough['Latitude'],
                                   longitudes=scarborough['Longitude'] );

Let's view the dataframe and the shape

In [24]:
print(scarborough_nearby_venues.shape)
scarborough_nearby_venues.head()

(90, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Let's check how many venues were returned for each neighborhood

In [25]:
scarborough_nearby_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",7,7,7,7,7,7
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


Let's find out how many unique categories can be curated from all the returned venues

In [26]:
print('There are {} uniques categories.'.format(len(scarborough_nearby_venues['Venue Category'].unique())))

There are 55 uniques categories.


Let's analyze each neighbourhood and get one hot encording

In [30]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_nearby_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,...,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's view the new dataframe shape and size.

In [31]:
scarborough_onehot.shape

(90, 56)

Group rows by neigbourhood and by the means of frequency of occurance of each category

In [34]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,...,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.0000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.20,0.0,0.000000,0.0000,0.0
1,"Birch Cliff, Cliffside West",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.0000,0.0
2,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.125000,0.0000,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000,0.000000,0.000000,0.083333,0.0,0.000000,0.0000,0.0,...,0.083333,0.166667,0.0,0.083333,0.000000,0.00,0.0,0.083333,0.0000,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0000,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0000,0.2
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.0000,0.2,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.1,0.000000,0.0000,0.0
7,"Guildwood, Morningside, West Hill",0.0,0.000,0.000000,0.000000,0.142857,0.0,0.142857,0.0000,0.0,...,0.000000,0.000000,0.0,0.142857,0.000000,0.00,0.0,0.000000,0.0000,0.0
8,"Kennedy Park, Ionview, East Birchmount Park",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0000,0.0
9,"Malvern, Rouge",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0000,0.0


In [35]:
scarborough_grouped.shape

(16, 56)

We can put this into a dataframe. 
But first, we will write a function to sort the venues in descending order.

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
Create a new dataframe that has the top 10 venues for each neigbourhood

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,College Stadium
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Fried Chicken Joint,Bank,Gas Station,Italian Restaurant,Noodle House,Pharmacy,Rental Car Location,Fast Food Restaurant,Thai Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Athletics & Sports,Auto Garage,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


#### Finally Cluster Neighborhood

But first let's create k-means to cluster the neigbourhood into 4 clusters

In [39]:
# set number of clusters
kclusters = 4

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 3], dtype=int32)

In [40]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough

# merge the data grouped with scarborough_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Bar,Construction & Landscaping,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Mexican Restaurant,Electronics Store,Bank,Rental Car Location,Breakfast Spot,Intersection,Medical Center,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
22,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,College Stadium


Finally we can visualize the resulting map using folium

In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys))) 
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    if(cluster != cluster):
        cluster =1.0; # Offset the odd cluster since we are using 4 clusters for 17 rows.
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We can examine the first two clusters with labels 0 and 1 :

In [61]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Woburn,0.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [62]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,"Guildwood, Morningside, West Hill",1.0,Mexican Restaurant,Electronics Store,Bank,Rental Car Location,Breakfast Spot,Intersection,Medical Center,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant
26,Cedarbrae,1.0,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,College Stadium
38,"Kennedy Park, Ionview, East Birchmount Park",1.0,Discount Store,Department Store,Convenience Store,Bus Station,Coffee Shop,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint
44,"Golden Mile, Clairlea, Oakridge",1.0,Bus Line,Bakery,Intersection,Ice Cream Shop,Bus Station,Park,Metro Station,Soccer Field,Electronics Store,Discount Store
51,"Cliffside, Cliffcrest, Scarborough Village West",1.0,American Restaurant,Motel,Athletics & Sports,Auto Garage,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
58,"Birch Cliff, Cliffside West",1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
65,"Dorset Park, Wexford Heights, Scarborough Town...",1.0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Athletics & Sports,Fast Food Restaurant
71,"Wexford, Maryvale",1.0,Middle Eastern Restaurant,Auto Garage,Bakery,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant
78,Agincourt,1.0,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant
82,"Clarks Corners, Tam O'Shanter, Sullivan",1.0,Pizza Place,Fried Chicken Joint,Bank,Gas Station,Italian Restaurant,Noodle House,Pharmacy,Rental Car Location,Fast Food Restaurant,Thai Restaurant
